Basics
------
This covers a fundamental use-case for DN<sup>3</sup>. Here, the TIDNet architecture from *Kostas & Rudzicz 2020
(https://doi.org/10.1088/1741-2552/abb7a7)* is evaluated using the Physionet motor dataset in a
*"leave multiple subjects out"* (aka person-stratified cross validation) training procedure.
This means, the number of subjects are divided into approximately evenly numbered sets, and the test performance of
each set is evaluated. The remaining sets for each test will be used to develop a neural network classifier.

Here we will focus on classifying imagined hand vs. foot movement, which are the runs 6, 10 and 14. So create a
configuration for the configuratron that specifies a single dataset, the length of the trials to cut for the dataset
 (6s), the events we are looking for (T1 and T2), and then to exclude all those sessions that are not 6, 10 and 14
(the sessions by default are listed as "S{subject number}R{run number}.edf". Finally, due to some issues with a few
people's recordings in the dataset, we can ignore those troublemakers. The following is the contents of a file named
"my_config.yml".

```yaml
Configuratron:
  preload: True

use_gpu: False

mmidb:
  name: "Physionet MMIDB"
  toplevel: /path/to/eegmmidb
  tmin: 0
  tlen: 6
  data_max: 0.001
  data_min: -0.001
  events:
    - T1
    - T2
  exclude_sessions:
    - "*R0[!48].edf"  # equivalently "*R0[1235679].edf"
    - "*R1[!2].edf"   # equivalently "*R1[134].edf"
  exclude_people:
    - S088
    - S090
    - S092
    - S100
  train_params:
    epochs: 7
    batch_size: 4
  lr: 0.0001
  folds: 5
```

Notice that in addition to the dataset and special `Configuratron` section we also include a `train_params` with the
dataset.
This last part is definitely *not mandatory*, it is an arbitrarily named additional set of configuration values that
will be used in our experiment, put there for convenience. See it in action below, but if you don't like it,
don't use it.

In [1]:
from dn3.configuratron import ExperimentConfig
from dn3.trainable.processes import StandardClassification
from dn3.trainable.models import TIDNet

# Since we are doing a lot of loading, this is nice to suppress some tedious information
import mne
mne.set_log_level(False)

First things first, we use `ExperimentConfig`, and the subsequently constructed `DatasetConfig` to rapidly construct
our `dataset`.

In [2]:
config_filename = 'my_config.yml'
experiment = ExperimentConfig(config_filename)
ds_config = experiment.datasets['mmidb']

dataset = ds_config.auto_construct_dataset()

Adding additional configuration entries: dict_keys(['train_params', 'folds', 'lr'])
Configuratron found 1 datasets.


Scanning /Volumes/Data/MMI. If there are a lot of files, this may take a while...: 100%|██████████| 4/4 [00:00<00:00,  5.92it/s, extension=.gdf]


Creating dataset of 315 Preloaded Epoched recordings from 105 people.


Loading Physionet MMIDB: 100%|██████████| 105/105 [00:17<00:00,  5.94person/s]


>> Physionet MMIDB | DSID: None | 105 people | 4408 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Constructed 1 channel maps
Used by 315 recordings:
EEG (original(new)): Fc5.(FC5) Fc3.(FC3) Fc1.(FC1) Fcz.(FCZ) Fc2.(FC2) Fc4.(FC4) Fc6.(FC6) C5..(C5) C3..(C3) C1..(C1) Cz..(CZ) C2..(C2) C4..(C4) C6..(C6) Cp5.(CP5) Cp3.(CP3) Cp1.(CP1) Cpz.(CPZ) Cp2.(CP2) Cp4.(CP4) Cp6.(CP6) Fp1.(FP1) Fpz.(FPZ) Fp2.(FP2) Af7.(AF7) Af3.(AF3) Afz.(AFZ) Af4.(AF4) Af8.(AF8) F7..(F7) F5..(F5) F3..(F3) F1..(F1) Fz..(FZ) F2..(F2) F4..(F4) F6..(F6) F8..(F8) Ft7.(FT7) Ft8.(FT8) T7..(T7) T8..(T8) T9..(T9) T10.(T10) Tp7.(TP7) Tp8.(TP8) P7..(P7) P5..(P5) P3..(P3) P1..(P1) Pz..(PZ) P2..(P2) P4..(P4) P6..(P6) P8..(P8) Po7.(PO7) Po3.(PO3) Poz.(POZ) Po4.(PO4) Po8.(PO8) O1..(O1) Oz..(OZ) O2..(O2) Iz..(IZ) 
EOG (original(new)): 
REF (original(new)): 
EXTRA (original(new)): 
Heuristically Assigned: Fc5.(FC5)  Fc3.(FC3)  Fc1.(FC1)  Fcz.(FCZ)  Fc2.(FC2)  Fc4.(FC4)  Fc6.(FC6)  C5..(C5)  C3..(C3)  C1..(C1)  Cz.

Let's create a function that makes a new model for each set of training people and a trainable process for
`StandardClassification`. The `cuda` argument handles whether the GPU is used to train the neural network if a
cuda-compatible PyTorch installation is operational.

In [3]:
def make_model_and_process():
    tidnet = TIDNet.from_dataset(dataset)
    return StandardClassification(tidnet, cuda=experiment.use_gpu, learning_rate=ds_config.lr)

That's pretty much it! We use a helper that initializes a TIDNet from any `Dataset/Thinker/EpochedRecording` and wrap
it with a `StandardClassification` process. Invoking this process will train the classifier.
Have a look through all the `trainable.processes`, they can wrap the *same model* to learn in stages (e.g. some sort
 of fine-tuning procedure from a general model -- covered in `examples/finetuning.ipynb`).

Now, we loop through five folds, *leaving multiple subjects out* (`dataset.lmso()`), fit the classifier,
then check the test results.

In [4]:
vars(ds_config.train_params)

{'epochs': 7, 'batch_size': 4}

In [5]:
results = []

for training, validation, test in dataset.lmso(ds_config.folds):
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation, **ds_config.train_params)

    results.append(process.evaluate(test)['Accuracy'])

print(results)
print("Average accuracy: {:.2%}".format(sum(results)/len(results)))

Training:   >> Physionet MMIDB | DSID: None | 63 people | 2646 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Validation: >> Physionet MMIDB | DSID: None | 21 people | 880 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Test:       >> Physionet MMIDB | DSID: None | 21 people | 882 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Creating TIDNet using: 90 channels x 1536 samples at 256Hz | 2 targets
Apple M-series GPU detected: training and model execution will be performed on MPS.


/Users/mojtaba/Library/CloudStorage/OneDrive-UniversityofWaterloo/Think/Time Management/UWaterloo/Research/my research/BENDR/env/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loading data with 0 additional workers


Epoch:  14%|#4        | 1/7 [00:00<?, ?epoch/s]

Iteration:   0%|          | 1/661 [00:00<?, ?batches/s]

...

Training: End of Epoch 7 | Accuracy: 92.32% | loss: 0.200 | lr: 2.565e-06 | momentum: 0.947 | epoch: 7.000 |


Predicting:   0%|          | 0/221 [00:00<?, ?it/s]

Validation: End of Epoch 7 | Accuracy: 85.71% | loss: 0.360 |
Loading best model...


Predicting:   0%|          | 0/880 [00:00<?, ?it/s]

[0.9036281108856201, 0.8764172196388245, 0.9229024648666382, 0.8356009125709534, 0.8602272868156433]
Average accuracy: 87.98%


Check out how we passed the train_params to `.fit()`, we could specify more arguments for `.fit()` by just adding them
to this section in the configuration.

Say we wanted to *instead*, get the performance of *each* person in the test fold independently. We could replace the
code above with a very simple alternative, that *leaves one subject out* or `.loso()`, that specifically.
It would look like:

In [ ]:
results = dict()
for training, validation, test in dataset.lmso(ds_config.folds):
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation,
                epochs=ds_config.train_params.epochs,
                batch_size=ds_config.train_params.batch_size)

    for _, _, test_thinker in test.loso():
        results[test_thinker.person_id] = process.evaluate(test_thinker)

print(results)

Training:   >> Physionet MMIDB | DSID: None | 63 people | 2646 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Validation: >> Physionet MMIDB | DSID: None | 21 people | 880 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Test:       >> Physionet MMIDB | DSID: None | 21 people | 882 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Creating TIDNet using: 90 channels x 1536 samples at 256Hz | 2 targets
Apple M-series GPU detected: training and model execution will be performed on MPS.
Loading data with 0 additional workers


Epoch:  14%|#4        | 1/7 [00:00<?, ?epoch/s]

Iteration:   0%|          | 1/661 [00:00<?, ?batches/s]

Training: End of Epoch 1 | Accuracy: 66.94% | loss: 0.666 | lr: 3.539e-05 | momentum: 0.917 | epoch: 1.000 |


Predicting:   0%|          | 0/220 [00:00<?, ?it/s]

...

Predicting:   0%|          | 0/42 [00:00<?, ?it/s]

Training:   >> Physionet MMIDB | DSID: None | 19 people | 796 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Validation: Person S108 - 42 trials | 0 transforms
Test:       Person S109 - 42 trials | 0 transforms


Predicting:   0%|          | 0/42 [00:00<?, ?it/s]

{'S001': OrderedDict({'Accuracy': 0.9285714030265808, 'loss': 0.2587059438228607}), 'S002': OrderedDict({'Accuracy': 0.976190447807312, 'loss': 0.17190757393836975}), 'S003': OrderedDict({'Accuracy': 1.0, 'loss': 0.08486204594373703}), 'S004': OrderedDict({'Accuracy': 0.9285714030265808, 'loss': 0.13500741124153137}), 'S005': OrderedDict({'Accuracy': 0.9523809552192688, 'loss': 0.19521886110305786}), 'S006': OrderedDict({'Accuracy': 1.0, 'loss': 0.10267361998558044}), 'S007': OrderedDict({'Accuracy': 1.0, 'loss': 0.06416495144367218}), 'S008': OrderedDict({'Accuracy': 0.8571428656578064, 'loss': 0.3228107988834381}), 'S009': OrderedDict({'Accuracy': 0.8809523582458496, 'loss': 0.1753239929676056}), 'S010': OrderedDict({'Accuracy': 0.7857142686843872, 'loss': 0.5850896835327148}), 'S011': OrderedDict({'Accuracy': 0.8809523582458496, 'loss': 0.20510387420654297}), 'S012': OrderedDict({'Accuracy': 0.9523809552192688, 'loss': 0.11980809271335602}), 'S013': OrderedDict({'Accuracy': 0.880952

TypeError: unsupported operand type(s) for +: 'int' and 'collections.OrderedDict'

In [12]:
avg_acc = sum(v['Accuracy'] for v in results.values()) / len(results)
print("Average accuracy: {:.2%}".format(avg_acc))

Average accuracy: 87.91%


Try filling in your own values to `my_config.yml` to run these examples on your next read through.